### Function mapping using *adaptive* package
In some cases the relation between the input variables and the output cannot be expressed analytically 
and only a numerical solution is possible. The process of obtaining an output for a wide range of input values and possibly creating an interpolation function is called function mapping. 

Considering that such relation can be presented in form of a callable function, a dedicated *adaptive* package provides a convenient way to perform this task. The package is based on the Bayesian optimization algorithm and is designed to work with functions that are expensive to evaluate. The package offers functionality to effectively map 1D and 2D functions and offers a possibility to extend it to higher dimensions. 

The function map can be used for creating an interpolation function for obtaining function values that were not calculated directly.

In the following example, a typical usage routine of the *adaptive* package for mapping the function is demonstrated. It is directly applied to the Reaction-Diffusion Equation case to reveal relation between growth rate profile features and  process parameters $p_o$ and $\tau_r$. Only 2D mapping (using Learner2D) is supported.

In [ ]:
import os, time
from math import sqrt
from copy import deepcopy
import numpy as np

import adaptive
import matplotlib.pyplot as plt
from backend import adaptive_tools
from backend.processclass2 import Experiment1D_Dimensionless

%matplotlib notebook
adaptive.notebook_extension()


## 1. Define the function to be learned
Here three various functions are defined to demonstrate the mapping process. They can be simply swapped to observe the mapping process live.


The first one is a simple exponential function

In [ ]:
def exp_func(xy):
    time.sleep(0.05)
    x, y = xy
    return np.exp(x/2) - x / np.exp(y/5)

The second one is a super-Gaussian function

In [ ]:
def super_gauss(xy):
    time.sleep(0.01)
    x, y = xy
    R = sqrt(x**2+y**2)
    s = 2
    n = 5
    return 1/s/sqrt(2*np.pi) *np.exp(-(R**2/2/s**2)**n)

Our target function is the relation between the indented growth rate profile peak position `r_max_n` and the process parameters `p_o` and `tau_r` that is obtained by solving the RDE equation of the Continuum model.
Here we use the `Experiment1D_Dimensionless` class that represents the dimensionless version of the RDE equation to obtain the solution.
First, we need to set up the RDE solver with necessary parameters:

In [ ]:
pr = Experiment1D_Dimensionless()
pr.beam_type = 'gauss'
pr.f0 = 1e6
pr.fwhm = 500
pr.step = 2
pr.order = 1

Then, we define a simple function that can receive the input parameters, set them into the RDE solver and return the result:

In [ ]:
def rde_r_max(xy):
    global pr
    _ = Experiment1D_Dimensionless() # this is needed to initialize internal variables
    pr = deepcopy(pr)
    x, y = xy
    pr.p_o = x
    pr.tau_r = y
    pr.solve_steady_state()
    return pr.r_max_n

## 2. Define the domain of the function

Define the extent of the domain for each variable that the fucntion will be learned in.

Here the $p_o$ is associated with x-axis and is limited between 0 and 20.

The $\tau_r$ is associated with y-axis and is limited between 1.0001 and 10000.

In [ ]:
bounds = ((1e-6, 3), (1.0001, 2000))

## 3. Learn the function

The learning process is started as soon as the `runner` object is created. It will show learning progress, number of points evaluated and current loss value.

The criteria used to finish the learning process is the *loss goal*. 
The smaller the value, the more accurate is the learned function. However, the smaller the value, the more time is needed to learn the function.

The number of tasks (engaged CPU cores) is limited to 8, by default all CPU cores are used.

In [ ]:
filename = 'test_mapping.int'
# Create the learner and provide it the function to be learned and the domain limits (bounds)
learner = adaptive.Learner2D(function=exp_func, bounds=bounds)
# Launch the learning process
runner = adaptive.Runner(learner, loss_goal=0.01, ntasks=4)
# It is possible to enable periodic saving of the learning result
# runner.start_periodic_saving(save_kwargs=dict(fname=filename), interval=60)
runner.live_info()
runner.live_plot(update_interval=0.5)

The result can be quicly plotted from the learner itself

In [ ]:
learner.plot(n=500, tri_alpha=0.2)

If any issues occur, the traceback can be displayed:

In [ ]:
for point, tb in runner.tracebacks:
    print(f"point: {point}:\n {tb}")


## 4. Inspect the learned function

Plot a large map of the learned function to see how well function features are captured.

If the accuracy is not satisfactory, the learning process can be continued by lowering the loss goal value.

In [ ]:
def plot(learner, npoints=300, tri_alpha=0.2, width=300, height=300, xlim=None, ylim=None):
    plot = learner.plot(npoints, tri_alpha=tri_alpha)
    if xlim is not None:
        plot.opts(xlim=xlim)
    if ylim is not None:
        plot.opts(ylim=ylim)
    plot.opts(width=width, height=height)
    return plot

In [ ]:
plot(learner, npoints=1500, tri_alpha=0.2, width=800, height=800)

## 5. Evaluate the learned function

Now the interpolation function is ready, it can be used to quickly assess the function value at any point in the domain without needing to solve the RDE.

It can be done in two ways:
* By directly creating an evenly spaced grid of x, y, z values just by specifying the number of points in a dimension:

In [ ]:
x, y, z = learner.interpolated_on_grid(2000)

* By extracting the interpolation function and specifying the x, y values manually:

In [ ]:
import numpy as np
from backend.adaptive_tools import learner_interpolator

interpolator = learner_interpolator(learner)

xi = np.linspace(0, 3, 200)
yi = np.linspace(1.0001, 2000, 200)
xx, yy = np.meshgrid(xi, yi)
zz = interpolator(xx, yy)

Keep in mind that the first method generates 1D arrays of x, y, while in the second method the xx and yy grids are 2D arrays.

## 6. Save the learned function

The learned function can be saved to a file and used later for interpolation.

In [ ]:
learner.save(filename)

## 7. Load the learned function

The saved learned function can be loaded from a file and used for interpolation.

In [ ]:
from backend.adaptive_tools import learner_load_full

learner_loaded = learner_load_full(filename)
learner_loaded.plot(n=500, tri_alpha=0.2)
interpolator = learner_interpolator(learner_loaded)

## 8. Learning ina transformed space
In some cases the learned function may have strong non-linear behavior (exponential, quadratic etc), whereas the learning algorithm chooses points based on the linear interpolation, making learning process inefficient. In such cases, it is beneficial to first transform the domain into a linear space, learn the function and then transform the learned function back to the original space.

Consider again the example with the RDE equation. By plotting the data from `examples\r_max_interp_1.0.int` file it becomes evident that the function has high variability in the lower-x part of the domain and low variability in the upper part:

In [ ]:
fname_example = r'../examples/r_max_interp_1.0.int'
learner_example = adaptive_tools.learner_load_full(fname_example)
learner_example.plot(n=500, tri_alpha=0.2)

Such behaviour prompts for a transformation of the x-axis. In this case, the logarithmic transformation is applied to the x-axis. The transformation is applied to the domain limits, the function itself and the interpolation function.
To apply initial transformation to the logarithmic space, the bounds and the function need to be modified.
The logarithm has to be applied to the x-bounds:

In [ ]:
xmin = np.log10(bounds[0][0])
xmax = np.log10(bounds[0][1])

bounds_logx = ((xmin, xmax), bounds[1])
bounds_logx

Now the learner will be looking at the transformed logarithmic space when choosing the next point for evaluation.

Next, the learner sends the x and y values to the function. Here the x value has to be transformed back before actual evaluation:

In [ ]:
def rde_r_max_logx(xy):
    global pr
    _ = Experiment1D_Dimensionless() # this is needed to initialize internal variables
    pr = deepcopy(pr)
    logx, y = xy
    x = 10**logx
    pr.p_o = x
    pr.tau_r = y
    pr.solve_steady_state()
    return pr.r_max_n

After that the learner can be started in the same manner as it ws shown above:

In [ ]:
filename = 'test_mapping_logx.int'
# Create the learner and provide it the function to be learned and the domain limits (bounds)
learner_logx = adaptive.Learner2D(function=exp_func, bounds=bounds_logx)
# Launch the learning process
runner_logx = adaptive.Runner(learner_logx, loss_goal=0.01, ntasks=1)
# It is possible to enable periodic saving of the learning result
# runner.start_periodic_saving(save_kwargs=dict(fname=filename), interval=60)
runner_logx.live_info()
runner_logx.live_plot(update_interval=0.5)

After the learning process is finished, the interpolation function can be transformed back to the original space:

In [ ]:
data = adaptive_tools.learner_data_to_numpy(learner_logx)
x = 10**data[:, 0]
data[:, 0] = x
learner_logx.data = data
learner_logx.bounds = bounds
learner_logx.plot(n=500, tri_alpha=0.2)